# Setting up 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/My\ Drive/

/content/drive/My Drive


In [3]:
%cd KaggleDataStorm

/content/drive/My Drive/KaggleDataStorm


In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "ds1047"
os.environ['KAGGLE_KEY'] = "26ca78d45179f04387b81f35ba031660"

!kaggle competitions download -c data-storm-10

  0% 0.00/213k [00:00<?, ?B/s]
100% 213k/213k [00:00<00:00, 28.8MB/s]
  0% 0.00/24.7k [00:00<?, ?B/s]
100% 24.7k/24.7k [00:00<00:00, 7.28MB/s]
  0% 0.00/825k [00:00<?, ?B/s]
100% 825k/825k [00:00<00:00, 53.4MB/s]
  0% 0.00/660k [00:00<?, ?B/s]
100% 660k/660k [00:00<00:00, 88.7MB/s]


# Import Libraries & Data

In [0]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from matplotlib import mlab
import seaborn as sns
import warnings

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

warnings.filterwarnings('ignore')
%matplotlib inline

In [0]:
train=pd.read_csv("credit_card_default_train.csv.zip")
test=pd.read_csv("credit_card_default_test.csv")

In [201]:
train.shape

(24000, 25)

**Twenty four thousand data points**

# Exploratory Data Analysis

In [0]:
cat_features = ['Balance_Limit_V1', 'EDUCATION_STATUS','AGE']
binary_features=['Gender','MARITAL_STATUS' ]
due_features = ['DUE_AMT_JULY', 'DUE_AMT_AUG', 'DUE_AMT_SEP', 'DUE_AMT_OCT', 'DUE_AMT_NOV', 'DUE_AMT_DEC']
paid_features = ['PAID_AMT_JULY', 'PAID_AMT_AUG', 'PAID_AMT_SEP', 'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC']
pay_int = ['PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV', 'PAY_DEC']

In [203]:
# Number of unique classes in each object column
train[cat_features].nunique()

Balance_Limit_V1    8
EDUCATION_STATUS    3
AGE                 4
dtype: int64

In [204]:
train['Balance_Limit_V1'].value_counts()

1M       5951
200K     5159
100K     3449
400K     3065
 500K    2790
300K     2411
1.5M     1138
2.5M       37
Name: Balance_Limit_V1, dtype: int64

In [205]:
train.groupby(['Balance_Limit_V1','NEXT_MONTH_DEFAULT']).count()

Client_ID  ...  PAID_AMT_DEC
Balance_Limit_V1 NEXT_MONTH_DEFAULT             ...              
 500K            0                        2289  ...          2289
                 1                         501  ...           501
1.5M             0                        1000  ...          1000
                 1                         138  ...           138
100K             0                        2207  ...          2207
                 1                        1242  ...          1242
1M               0                        5061  ...          5061
                 1                         890  ...           890
2.5M             0                          34  ...            34
                 1                           3  ...             3
200K             0                        3805  ...          3805
                 1                        1354  ...          1354
300K             0                        1805  ...          1805
                 1                         606  ...           606
400K             0                        2469  ...          2469
                 1                         596  ...           596

[16 rows x 23 columns]

In [206]:
train.groupby(['EDUCATION_STATUS','NEXT_MONTH_DEFAULT']).count()

Client_ID  ...  PAID_AMT_DEC
EDUCATION_STATUS NEXT_MONTH_DEFAULT             ...              
Graduate         0                        6828  ...          6828
                 1                        1650  ...          1650
High School      0                        2939  ...          2939
                 1                         986  ...           986
Other            0                        8903  ...          8903
                 1                        2694  ...          2694

[6 rows x 23 columns]

# Feature Engineering

### Categorical Variable Preprocessing

In [0]:
train_test = pd.concat([train, test], axis=0)

In [253]:
train_test.head()

,AGE,Balance_Limit_V1,Client_ID,DUE_AMT_AUG,DUE_AMT_DEC,DUE_AMT_JULY,DUE_AMT_NOV,DUE_AMT_OCT,DUE_AMT_SEP,EDUCATION_STATUS,Gender,MARITAL_STATUS,NEXT_MONTH_DEFAULT,PAID_AMT_AUG,PAID_AMT_DEC,PAID_AMT_JULY,PAID_AMT_NOV,PAID_AMT_OCT,PAID_AMT_SEP,PAY_AUG,PAY_DEC,PAY_JULY,PAY_NOV,PAY_OCT,PAY_SEP
0,31-45,1M,A2,3389,-13982,3248,162772,39418,6004,Graduate,M,Other,0.0,6004,538165,3437,0,162772,39418,-1,-1,-1,-1,-1,-1
1,Less than 30,1M,A3,151818,124590,353351,80811,43530,26948,High School,F,Other,0.0,46200,33666,151818,942,80811,43530,-1,0,0,-1,-1,-1
2,31-45,100K,A4,16082,0,16681,0,0,15477,High School,F,Single,1.0,0,0,0,0,0,0,3,-2,4,-2,2,2
3,31-45,200K,A5,92848,102740,90457,100353,97309,95193,Graduate,F,Single,1.0,3890,3918,3855,4049,4620,3696,0,0,2,0,0,0
4,31-45,1M,A6,419466,453899,429556,445271,435354,429785,Graduate,F,Other,0.0,20790,17325,0,16401,17325,16170,2,0,2,0,0,0


In [0]:
months=["_JULY",	"_AUG",	"_SEP",	"_OCT",	"_NOV",	"_DEC"]

In [0]:
for month in months:
  train_test["DELAY"+month]=train_test["PAY"+month]

In [0]:
for feature in pay_int:
  train_test.loc[train_test[feature] <= 0,feature]=0

In [0]:
train_test['delayed_months'] = train_test[pay_int].sum(axis=1)

In [0]:
train_test.drop(columns=pay_int,inplace=True)

In [259]:
train_test.head()

,AGE,Balance_Limit_V1,Client_ID,DUE_AMT_AUG,DUE_AMT_DEC,DUE_AMT_JULY,DUE_AMT_NOV,DUE_AMT_OCT,DUE_AMT_SEP,EDUCATION_STATUS,Gender,MARITAL_STATUS,NEXT_MONTH_DEFAULT,PAID_AMT_AUG,PAID_AMT_DEC,PAID_AMT_JULY,PAID_AMT_NOV,PAID_AMT_OCT,PAID_AMT_SEP,DELAY_JULY,DELAY_AUG,DELAY_SEP,DELAY_OCT,DELAY_NOV,DELAY_DEC,delayed_months
0,31-45,1M,A2,3389,-13982,3248,162772,39418,6004,Graduate,M,Other,0.0,6004,538165,3437,0,162772,39418,-1,-1,-1,-1,-1,-1,0
1,Less than 30,1M,A3,151818,124590,353351,80811,43530,26948,High School,F,Other,0.0,46200,33666,151818,942,80811,43530,0,-1,-1,-1,-1,0,0
2,31-45,100K,A4,16082,0,16681,0,0,15477,High School,F,Single,1.0,0,0,0,0,0,0,4,3,2,2,-2,-2,11
3,31-45,200K,A5,92848,102740,90457,100353,97309,95193,Graduate,F,Single,1.0,3890,3918,3855,4049,4620,3696,2,0,0,0,0,0,2
4,31-45,1M,A6,419466,453899,429556,445271,435354,429785,Graduate,F,Other,0.0,20790,17325,0,16401,17325,16170,2,2,0,0,0,0,4


In [0]:
numerical_features = due_features+ paid_features+['delayed_months']

In [261]:
numerical_features

['DUE_AMT_JULY',
 'DUE_AMT_AUG',
 'DUE_AMT_SEP',
 'DUE_AMT_OCT',
 'DUE_AMT_NOV',
 'DUE_AMT_DEC',
 'PAID_AMT_JULY',
 'PAID_AMT_AUG',
 'PAID_AMT_SEP',
 'PAID_AMT_OCT',
 'PAID_AMT_NOV',
 'PAID_AMT_DEC',
 'delayed_months']

In [0]:
train_test = pd.get_dummies(data=train_test, columns=["DELAY"+month for month in months])

In [0]:
train_test = pd.get_dummies(data=train_test, columns=binary_features, drop_first=True)

In [0]:
train_test = pd.get_dummies(data=train_test, columns=['AGE'])

In [0]:
train_test.drop(columns='AGE_More than 65',inplace=True)

In [0]:
train_test = pd.get_dummies(data=train_test, columns=['Balance_Limit_V1'])
train_test.drop(columns='Balance_Limit_V1_2.5M',inplace=True)

In [0]:
train_test = pd.get_dummies(data=train_test, columns=['EDUCATION_STATUS'])
train_test.drop(columns='EDUCATION_STATUS_High School',inplace=True)

In [0]:
months=["_JULY",	"_AUG",	"_SEP",	"_OCT",	"_NOV",	"_DEC"]

In [0]:
for month in months:
  train_test['PAID_DUE'+month]=train_test['PAID_AMT'+month]-train_test['DUE_AMT'+month]
train_test['AVG_PAID']=train_test[paid_features].mean(axis=1)

In [270]:
test_processed = train_test[train_test['NEXT_MONTH_DEFAULT'].isna()]
test_processed.shape

(6000, 100)

In [271]:
train_processed = train_test[train_test['NEXT_MONTH_DEFAULT'].notna()]
train_processed.shape

(24000, 100)

In [272]:
train_processed.head()

,Client_ID,DUE_AMT_AUG,DUE_AMT_DEC,DUE_AMT_JULY,DUE_AMT_NOV,DUE_AMT_OCT,DUE_AMT_SEP,NEXT_MONTH_DEFAULT,PAID_AMT_AUG,PAID_AMT_DEC,PAID_AMT_JULY,PAID_AMT_NOV,PAID_AMT_OCT,PAID_AMT_SEP,delayed_months,DELAY_JULY_-2,DELAY_JULY_-1,DELAY_JULY_0,DELAY_JULY_1,DELAY_JULY_2,DELAY_JULY_3,DELAY_JULY_4,DELAY_JULY_5,DELAY_JULY_6,DELAY_JULY_7,DELAY_JULY_8,DELAY_AUG_-2,DELAY_AUG_-1,DELAY_AUG_0,DELAY_AUG_1,DELAY_AUG_2,DELAY_AUG_3,DELAY_AUG_4,DELAY_AUG_5,DELAY_AUG_6,DELAY_AUG_7,DELAY_AUG_8,DELAY_SEP_-2,DELAY_SEP_-1,DELAY_SEP_0,...,DELAY_NOV_-1,DELAY_NOV_0,DELAY_NOV_2,DELAY_NOV_3,DELAY_NOV_4,DELAY_NOV_5,DELAY_NOV_6,DELAY_NOV_7,DELAY_NOV_8,DELAY_DEC_-2,DELAY_DEC_-1,DELAY_DEC_0,DELAY_DEC_2,DELAY_DEC_3,DELAY_DEC_4,DELAY_DEC_5,DELAY_DEC_6,DELAY_DEC_7,DELAY_DEC_8,Gender_M,MARITAL_STATUS_Single,AGE_31-45,AGE_46-65,AGE_Less than 30,Balance_Limit_V1_ 500K,Balance_Limit_V1_1.5M,Balance_Limit_V1_100K,Balance_Limit_V1_1M,Balance_Limit_V1_200K,Balance_Limit_V1_300K,Balance_Limit_V1_400K,EDUCATION_STATUS_Graduate,EDUCATION_STATUS_Other,PAID_DUE_JULY,PAID_DUE_AUG,PAID_DUE_SEP,PAID_DUE_OCT,PAID_DUE_NOV,PAID_DUE_DEC,AVG_PAID
0,A2,3389,-13982,3248,162772,39418,6004,0.0,6004,538165,3437,0,162772,39418,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,189,2615,33414,123354,-162772,552147,124966.000000
1,A3,151818,124590,353351,80811,43530,26948,0.0,46200,33666,151818,942,80811,43530,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-201533,-105618,16582,37281,-79869,-90924,59494.500000
2,A4,16082,0,16681,0,0,15477,1.0,0,0,0,0,0,0,11,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,-16681,-16082,-15477,0,0,0,0.000000
3,A5,92848,102740,90457,100353,97309,95193,1.0,3890,3918,3855,4049,4620,3696,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,-86602,-88958,-91497,-92689,-96304,-98822,4004.666667
4,A6,419466,453899,429556,445271,435354,429785,0.0,20790,17325,0,16401,17325,16170,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,-429556,-398676,-413615,-418029,-428870,-436574,14668.500000


### Continuous Variable Preprocessing

In [0]:
for feature in due_features:
  train_processed[feature]=train_processed[feature].apply( lambda x: np.log1p(x) if (x>0) else 0 )
  test_processed[feature]=test_processed[feature].apply( lambda x: np.log1p(x) if (x>0) else 0 )
  scaler = StandardScaler().fit(np.array(train_processed[feature]).reshape(-1, 1))
  train_processed[feature] = scaler.transform(np.array(train_processed[feature]).reshape(-1, 1))
  test_processed[feature] = scaler.transform(np.array(test_processed[feature]).reshape(-1, 1))

In [0]:
for feature in paid_features:
  train_processed[feature]=train_processed[feature].apply( lambda x: np.log1p(x) if (x>0) else 0 )
  test_processed[feature]=test_processed[feature].apply( lambda x: np.log1p(x) if (x>0) else 0 )
  scaler = StandardScaler().fit(np.array(train_processed[feature]).reshape(-1, 1))
  train_processed[feature] = scaler.transform(np.array(train_processed[feature]).reshape(-1, 1))
  test_processed[feature] = scaler.transform(np.array(test_processed[feature]).reshape(-1, 1))

In [0]:
for month in months:
  train_processed["PAID_DUE"  + month]=train_processed["PAID_DUE"  + month].apply( lambda x: np.log1p(x) if (x>0) else 0 )
  test_processed["PAID_DUE"  + month]=test_processed["PAID_DUE"  + month].apply( lambda x: np.log1p(x) if (x>0) else 0 )

In [0]:
 for month in months:
  scaler = StandardScaler().fit(np.array(train_processed["PAID_DUE"  + month]).reshape(-1, 1))
  train_processed["PAID_DUE"  + month] = scaler.transform(np.array(train_processed["PAID_DUE"  + month]).reshape(-1, 1))
  test_processed["PAID_DUE"  + month] = scaler.transform(np.array(test_processed["PAID_DUE"  + month]).reshape(-1, 1))

In [283]:
test_processed.head()

,Client_ID,DUE_AMT_AUG,DUE_AMT_DEC,DUE_AMT_JULY,DUE_AMT_NOV,DUE_AMT_OCT,DUE_AMT_SEP,NEXT_MONTH_DEFAULT,PAID_AMT_AUG,PAID_AMT_DEC,PAID_AMT_JULY,PAID_AMT_NOV,PAID_AMT_OCT,PAID_AMT_SEP,delayed_months,DELAY_JULY_-2,DELAY_JULY_-1,DELAY_JULY_0,DELAY_JULY_1,DELAY_JULY_2,DELAY_JULY_3,DELAY_JULY_4,DELAY_JULY_5,DELAY_JULY_6,DELAY_JULY_7,DELAY_JULY_8,DELAY_AUG_-2,DELAY_AUG_-1,DELAY_AUG_0,DELAY_AUG_1,DELAY_AUG_2,DELAY_AUG_3,DELAY_AUG_4,DELAY_AUG_5,DELAY_AUG_6,DELAY_AUG_7,DELAY_AUG_8,DELAY_SEP_-2,DELAY_SEP_-1,DELAY_SEP_0,...,DELAY_NOV_-1,DELAY_NOV_0,DELAY_NOV_2,DELAY_NOV_3,DELAY_NOV_4,DELAY_NOV_5,DELAY_NOV_6,DELAY_NOV_7,DELAY_NOV_8,DELAY_DEC_-2,DELAY_DEC_-1,DELAY_DEC_0,DELAY_DEC_2,DELAY_DEC_3,DELAY_DEC_4,DELAY_DEC_5,DELAY_DEC_6,DELAY_DEC_7,DELAY_DEC_8,Gender_M,MARITAL_STATUS_Single,AGE_31-45,AGE_46-65,AGE_Less than 30,Balance_Limit_V1_ 500K,Balance_Limit_V1_1.5M,Balance_Limit_V1_100K,Balance_Limit_V1_1M,Balance_Limit_V1_200K,Balance_Limit_V1_300K,Balance_Limit_V1_400K,EDUCATION_STATUS_Graduate,EDUCATION_STATUS_Other,PAID_DUE_JULY,PAID_DUE_AUG,PAID_DUE_SEP,PAID_DUE_OCT,PAID_DUE_NOV,PAID_DUE_DEC,AVG_PAID
0,A20170,0.702939,0.928066,0.685488,0.911534,0.856096,0.602091,NaN,0.616007,0.747359,0.617043,0.812381,1.113084,1.425627,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,1,-0.399482,-0.406524,3.199518,-0.384413,-0.395253,-0.408724,50953.333333
1,A16887,0.486556,0.626120,0.454111,0.589247,0.550623,0.514244,NaN,0.279900,0.479986,0.251952,0.462505,0.455822,0.357692,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,-0.399482,-0.406524,-0.392124,-0.384413,-0.395253,-0.408724,4224.666667
2,A3430,-0.119760,-0.482827,-0.549290,0.008045,0.008571,-0.095931,NaN,-2.023288,1.038616,0.424654,0.278505,-1.813191,0.215684,2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,2.278385,-0.406524,-0.392124,-0.384413,-0.395253,2.913685,8601.666667
3,A3696,-0.256887,-2.141638,-2.829255,-2.288882,-2.376534,-0.211924,NaN,0.393596,-1.702596,0.373244,-1.778211,-1.813191,-1.895539,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,1,0,2.377659,-0.406524,-0.392124,-0.384413,-0.395253,-0.408724,1925.000000
4,A20474,-2.607238,-2.141638,0.447888,-2.288882,-2.376534,-2.485717,NaN,-2.023288,-1.702596,-2.069420,-1.778211,-1.813191,-1.895539,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-0.399482,-0.406524,-0.392124,-0.384413,-0.395253,-0.408724,0.000000


In [0]:
sclaer=MinMaxScaler().fit(np.array(train_processed['AVG_PAID']).reshape(-1, 1))
train_processed["AVG_PAID" ] = scaler.transform(np.array(train_processed["AVG_PAID"]).reshape(-1, 1))
test_processed["AVG_PAID" ] = scaler.transform(np.array(test_processed["AVG_PAID" ]).reshape(-1, 1))

In [0]:
train_processed.drop(columns='Client_ID',inplace=True)

# Modelling

In [0]:
X=train_processed.drop(columns='NEXT_MONTH_DEFAULT')
y=train_processed['NEXT_MONTH_DEFAULT'].astype("int32")

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, shuffle=True,stratify=y)

In [0]:
from imblearn.over_sampling import SMOTE

X_resampled,y_resampled=SMOTE(sampling_strategy='auto', k_neighbors=3).fit_resample(X_train,y_train)

In [306]:
X.head()

,DUE_AMT_AUG,DUE_AMT_DEC,DUE_AMT_JULY,DUE_AMT_NOV,DUE_AMT_OCT,DUE_AMT_SEP,PAID_AMT_AUG,PAID_AMT_DEC,PAID_AMT_JULY,PAID_AMT_NOV,PAID_AMT_OCT,PAID_AMT_SEP,delayed_months,DELAY_JULY_-2,DELAY_JULY_-1,DELAY_JULY_0,DELAY_JULY_1,DELAY_JULY_2,DELAY_JULY_3,DELAY_JULY_4,DELAY_JULY_5,DELAY_JULY_6,DELAY_JULY_7,DELAY_JULY_8,DELAY_AUG_-2,DELAY_AUG_-1,DELAY_AUG_0,DELAY_AUG_1,DELAY_AUG_2,DELAY_AUG_3,DELAY_AUG_4,DELAY_AUG_5,DELAY_AUG_6,DELAY_AUG_7,DELAY_AUG_8,DELAY_SEP_-2,DELAY_SEP_-1,DELAY_SEP_0,DELAY_SEP_1,DELAY_SEP_2,...,DELAY_NOV_-1,DELAY_NOV_0,DELAY_NOV_2,DELAY_NOV_3,DELAY_NOV_4,DELAY_NOV_5,DELAY_NOV_6,DELAY_NOV_7,DELAY_NOV_8,DELAY_DEC_-2,DELAY_DEC_-1,DELAY_DEC_0,DELAY_DEC_2,DELAY_DEC_3,DELAY_DEC_4,DELAY_DEC_5,DELAY_DEC_6,DELAY_DEC_7,DELAY_DEC_8,Gender_M,MARITAL_STATUS_Single,AGE_31-45,AGE_46-65,AGE_Less than 30,Balance_Limit_V1_ 500K,Balance_Limit_V1_1.5M,Balance_Limit_V1_100K,Balance_Limit_V1_1M,Balance_Limit_V1_200K,Balance_Limit_V1_300K,Balance_Limit_V1_400K,EDUCATION_STATUS_Graduate,EDUCATION_STATUS_Other,PAID_DUE_JULY,PAID_DUE_AUG,PAID_DUE_SEP,PAID_DUE_OCT,PAID_DUE_NOV,PAID_DUE_DEC,AVG_PAID
0,-0.401489,-2.141638,-0.495221,0.718398,0.328018,-0.201717,0.404446,1.710528,0.226929,-1.778211,1.413548,0.988865,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1.282880,2.075929,2.972926,3.496487,-0.395253,3.745937,39268.099358
1,0.630173,0.678302,0.858282,0.542917,0.353378,0.192416,0.973799,0.993641,1.295145,0.040484,1.225262,1.015912,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-0.399482,-0.406524,2.746594,3.100361,-0.395253,-0.408724,18694.758394
2,0.020996,-2.141638,-0.022998,-2.288882,-2.376534,0.046843,-2.023288,-1.702596,-2.069420,-1.778211,-1.813191,-1.895539,11,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,-0.399482,-0.406524,-0.392124,-0.384413,-0.395253,-0.408724,-0.408724
3,0.496743,0.631957,0.464977,0.597193,0.558974,0.523706,0.283363,0.437367,0.259287,0.427482,0.455822,0.343752,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,-0.399482,-0.406524,-0.392124,-0.384413,-0.395253,-0.408724,1257.991843
4,0.905952,0.989032,0.914652,0.970589,0.941900,0.919419,0.750992,0.821817,-2.069420,0.798887,0.811189,0.745992,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,-0.399482,-0.406524,-0.392124,-0.384413,-0.395253,-0.408724,4608.925901


In [0]:
from sklearn.metrics import f1_score

In [307]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = {'n_estimators': [50,100,150,200,250,500],
               "max_features": ['auto', 'sqrt', 'log2'],
               'max_depth': [1,2,3,4,5,6,7,8,9],
               "criterion": ["gini", "entropy"]}

rf = RandomForestClassifier(class_weight="balanced")

rf_cv = RandomizedSearchCV(rf, param_distributions = param_dist, 
                           cv = 5, random_state=0, n_jobs = -1)

rf_cv.fit(X_resampled,y_resampled)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight='balanced',
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
         

In [308]:
print("Tuned Random Forest Parameters: %s" % (rf_cv.best_params_))

Tuned Random Forest Parameters: {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 9, 'criterion': 'gini'}


In [0]:

Ran = RandomForestClassifier(criterion= 'gini', max_depth= 9 ,
                                     max_features='sqrt', n_estimators= 200, 
                                     random_state=0)
Ran.fit(X_resampled,y_resampled)
y_pred = Ran.predict(X_test)


In [311]:
print(f1_score(y_test, y_pred, average='micro'))

0.7991666666666667


# NN

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
from sklearn.utils import class_weight
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [0]:
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)

    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive

    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [0]:
def macro_double_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    This version uses the computation of soft-F1 for both positive and negative class for each label.

    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)

    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    tn = tf.reduce_sum((1 - y_hat) * (1 - y), axis=0)
    soft_f1_class1 = 2*tp / (2*tp + fn + fp + 1e-16)
    soft_f1_class0 = 2*tn / (2*tn + fn + fp + 1e-16)
    # reduce 1 - soft-f1_class1 in order to increase soft-f1 on class 1
    cost_class1 = 1 - soft_f1_class1
    # reduce 1 - soft-f1_class0 in order to increase soft-f1 on class 0
    cost_class0 = 1 - soft_f1_class0
    # take into account both class 1 and class 0
    cost = 0.5 * (cost_class1 + cost_class0)
    macro_cost = tf.reduce_mean(cost)  # average on all labels
    return macro_cost

In [0]:
class_weights = class_weight.compute_class_weight("balanced",[0,1] ,y_train)

In [295]:
X_train.shape

(19200, 98)

In [0]:
metrics = [ tf.metrics.Precision(
    name="precision"), tf.metrics.Recall(name="recall"), macro_f1]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(90, activation='relu',
                                input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dense(45,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(tf.keras.layers.Dense(45, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid', use_bias=True))
LR = 1e-5
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=LR), loss=macro_double_soft_f1, metrics=metrics)

In [0]:
history = model.fit(X_resampled,y_resampled, epochs=500, batch_size=1000,shuffle=True,validation_split=0.1, verbose=True) #

In [0]:
y_pred=model.predict_classes(X_test)

In [304]:
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

0.7533333333333333
0.6811372770452053
0.7654713743204792


In [0]:
test_processed.head()

,Client_ID,DUE_AMT_AUG,DUE_AMT_DEC,DUE_AMT_JULY,DUE_AMT_NOV,DUE_AMT_OCT,DUE_AMT_SEP,NEXT_MONTH_DEFAULT,PAID_AMT_AUG,PAID_AMT_DEC,PAID_AMT_JULY,PAID_AMT_NOV,PAID_AMT_OCT,PAID_AMT_SEP,PAY_AUG,PAY_DEC,PAY_JULY,PAY_NOV,PAY_OCT,PAY_SEP,delayed_months,Balance_Limit_V1_1.5M,Balance_Limit_V1_100K,Balance_Limit_V1_1M,Balance_Limit_V1_2.5M,Balance_Limit_V1_200K,Balance_Limit_V1_300K,Balance_Limit_V1_400K,Gender_M,EDUCATION_STATUS_High School,EDUCATION_STATUS_Other,MARITAL_STATUS_Single,AGE_46-65,AGE_Less than 30,AGE_More than 65
0,A20170,-0.70,-0.24,-0.56,-0.54,-0.43,-0.86,nan,0.00,0.01,0.01,0.02,0.04,0.09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0
1,A16887,-0.79,-0.41,-0.64,-0.76,-0.60,-0.88,nan,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0
2,A3430,-0.86,-0.48,-0.70,-0.83,-0.67,-0.92,nan,0.00,0.03,0.00,0.00,0.00,0.00,0,0,0,0,0,2,2,0,0,0,0,0,0,1,1,0,1,0,0,1,0
3,A3696,-0.86,-0.48,-0.71,-0.84,-0.68,-0.93,nan,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0
4,A20474,-0.87,-0.48,-0.64,-0.84,-0.68,-0.93,nan,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0


In [0]:
test_processed_clinet=test_processed['Client_ID']

In [0]:
test_data=test_processed.drop(columns=['Client_ID','NEXT_MONTH_DEFAULT'])

In [0]:
X_train.shape

(19200, 33)

In [0]:
y_pred_test = grid2.best_estimator_.predict(test_data)

In [0]:
test_processed_clinet = test_processed_clinet.to_numpy()

In [0]:
data = {'Client_ID': test_processed_clinet, 'NEXT_MONTH_DEFAULT': y_pred_test}

In [0]:
results = pd.DataFrame(data)
results.head()

,Client_ID,NEXT_MONTH_DEFAULT
0,A20170,0
1,A16887,0
2,A3430,0
3,A3696,0
4,A20474,0


In [0]:
results.to_csv("results.csv", index=False)